In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import ensemble_model.preprocesser as preprocesser 
import ensemble_model.ensemble_model as em 
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer, RobertaModel, RobertaTokenizer

In [2]:

df = pd.read_csv(r'/root/autodl-tmp/dataset.csv', index_col=0, encoding='utf_8_sig')
df.fillna('', inplace=True)
label2id={'negative':0,'positive':1}
df = df.replace({"label": label2id})
df
df['command'] = df['diff'].apply(lambda x : ' '.join(x.split('@@')[:2]))
df
df_dataset = []
for index, row in df.iterrows():
  df_dataset.append([row['message'],row['command'],row['label']])

/tmp/ipykernel_1332/3675444814.py:4: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.replace({"label": label2id})


In [3]:
# Load BERT and CodeBERT models and tokenizers
bert_model = BertModel.from_pretrained('/root/autodl-tmp/models/bert-base-cased')
bert_tokenizer = BertTokenizer.from_pretrained('/root/autodl-tmp/models/bert-base-cased')

codebert_model = RobertaModel.from_pretrained('/root/autodl-tmp/models/codebert-base')
codebert_tokenizer = RobertaTokenizer.from_pretrained('/root/autodl-tmp/models/codebert-base')

/root/miniconda3/lib/python3.10/site-packages/torch/_utils.py:831: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return self.fget.__get__(instance, owner)()


In [4]:
# Split the dataset
# preprocesser = preprocesser.SentencePairDataset(train_data, bert_tokenizer, codebert_tokenizer)
train_data, val_data = train_test_split(df_dataset, test_size=0.2, random_state=42)

# Create Datasets and DataLoaders
train_dataset = preprocesser.SentencePairDataset(train_data, bert_tokenizer, codebert_tokenizer)
val_dataset = preprocesser.SentencePairDataset(val_data, bert_tokenizer, codebert_tokenizer)

train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)

# Initialize the model
model = em.CombinedModel(bert_model, codebert_model, bert_tokenizer, codebert_tokenizer)

# Train the model
model.trainer(train_loader, val_loader,num_epochs=10)

/root/miniconda3/lib/python3.10/site-packages/transformers/optimization.py:588: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/10, Loss: 0.38329278773595044
Epoch 2/10, Loss: 0.21524139500467895
Epoch 3/10, Loss: 0.11586756651115368
Epoch 4/10, Loss: 0.06922818381718236
Epoch 5/10, Loss: 0.046098551449112034
Epoch 6/10, Loss: 0.0346014918456492
Epoch 7/10, Loss: 0.03234461589072668
Epoch 8/10, Loss: 0.02589775442330885
Epoch 9/10, Loss: 0.0260394861467539
Epoch 10/10, Loss: 0.02616635528946839
Validation Accuracy: 0.9050914483440435
Precision: 0.8792207792207792
Recall: 0.8724226804123711
F1-Score: 0.8758085381630013
